In [122]:
from dash import Dash, html, dcc
import pandas as pd
import plotly.express as px
import numpy as np
app = Dash(__name__)

# App layout
app.layout = html.Div([
    html.Div(children='My First App with Data and a Graph'),
    #dash_table.DataTable(data=df.to_dict('records'), page_size=10),
    dcc.Graph(figure=px.histogram(data['content'], x='release_date', y='lexical_complexity_score', histfunc=None))
])
if __name__ == '__main__':
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


In [39]:
import http.client
import json

conn = http.client.HTTPSConnection("get-songs-from-interval-xg7jaquagq-tl.a.run.app")
payload = json.dumps({
  "action": "get",
  "resource": "songs_from_interval",
  "min": 1950,
  "max": 1970
})
headers = {
  'Content-Type': 'application/json'
}
conn.request("POST", "/?=null", payload, headers)
res = conn.getresponse()
data = json.loads(res.read().decode("utf-8"))
transform_into_float = lambda k, v: float(v) if k == 'lexical_complexity_score' else v
data['content'] = [{k : transform_into_float(k,v) for k, v in data.items()} for data in data['content']]

In [42]:
data['content']

[{'song_title': 'Não tem solução',
  'artist': 'Dorival Caymmi',
  'release_date': 1952,
  'rank_position': 22,
  'lexical_complexity_score': 0.6818182,
  'mxm_dataset_ID': 'TRPDNEY128F1472E46'},
 {'song_title': 'Sebastiana',
  'artist': 'Jackson do Pandeiro',
  'release_date': 1953,
  'rank_position': 58,
  'lexical_complexity_score': 0.6666667,
  'mxm_dataset_ID': 'TRJILUW128F92F8BFF'},
 {'song_title': 'Saudosa Maloca',
  'artist': 'Demônios da Garoa',
  'release_date': 1955,
  'rank_position': 2,
  'lexical_complexity_score': 0.5151515,
  'mxm_dataset_ID': 'TRCIOEX12903CB102E'},
 {'song_title': 'Saudade da Bahia',
  'artist': 'Dorival Caymmi',
  'release_date': 1957,
  'rank_position': 12,
  'lexical_complexity_score': 0.53333336,
  'mxm_dataset_ID': 'TRUATRA128F1475223'},
 {'song_title': 'Franqueza',
  'artist': 'Maysa',
  'release_date': 1957,
  'rank_position': 18,
  'lexical_complexity_score': 0.7105263,
  'mxm_dataset_ID': 'TRPYNGY12903CE2EA7'},
 {'song_title': 'Estrada do sol'

In [2]:
#import functions_framework
import pymysql
import json

def filter_songs_from_interval(conn, request_json):
  with conn:
    with conn.cursor() as cursor:          
      min_val = request_json['min']
      if request_json and 'max' in request_json:
        max_val = request_json['max']
        query = "SELECT * FROM brazilian_songs WHERE release_date >= {} AND release_date <= {}".format(min_val, max_val) 
      else:
        query = "SELECT * FROM brazilian_songs WHERE release_date >= {}".format(min_val)
      cursor.execute(query)
      songs = cursor.fetchall()
      song_list = []
      for song in songs: 
        dic = {}
        song_title = song['song_title']
        artist = song['artist']
        release_date = song['release_date']
        rank_position = song['rank_position']
        lexical_complexity_score = song['lexical_complexity_score']
        mxm_dataset_ID = song['mxm_dataset_ID']
        dic['song_title'] = song_title
        dic['artist'] = artist
        dic['release_date'] = release_date
        dic['rank_position'] = rank_position
        dic['lexical_complexity_score'] = lexical_complexity_score
        dic['mxm_dataset_ID'] = mxm_dataset_ID
        song_list.append(dic)
    return song_list

def connect_db():
  conn = pymysql.connect(
    host="34.95.238.213",
    port = 3306,
    user="root",
    password="", # to-do fill
    db="brazilian_songs_dataset",
    cursorclass=pymysql.cursors.DictCursor          
  )
  return conn  

#@functions_framework.http
def get_avg_complexity_by_year_in_interval(request):
  request_json = request#.get_json(silent=True) # UNDO!
  # Copy Below
  response = {}
  if request_json and 'resource' in request_json and request_json['resource'] == "get_average_lexical_complexity":
    if request_json and 'min' in request_json and 'timestamp' in request_json:
      try:
        conn = connect_db()
        song_list = filter_songs_from_interval(conn, request_json)        
        response['content'] = sorted(song_list, key=lambda x: x['release_date'])
        # Convert lexical complexity measure from string into float
        transform_into_float = lambda k, v: float(v) if k == 'lexical_complexity_score' else v          
        response['content'] = [{k : transform_into_float(k,v) for k, v in data.items()} for data in response['content']]
        timestamp = int(request_json['timestamp'])
        if 'max' in request_json:
          max_date = request_json['max']
        else:
          max_date = 2010
        min_date = request_json['min']
        dic = {}
        for obj in response['content']:
          if (obj['release_date'] - min_date) <= timestamp:
            if (min_date + timestamp) > max_date:
              key = "{},{}".format(min_date, max_date)
            else:
              key = "{},{}".format(min_date, min_date + timestamp)
            if not key in dic:
              dic[key] = [obj["lexical_complexity_score"]]
            else:
              dic[key].append(obj["lexical_complexity_score"])
          else:
            min_date = obj['release_date']
            if (min_date + timestamp) > max_date:
              key = "{},{}".format(min_date, max_date)
            else:
              key = "{},{}".format(min_date, min_date + timestamp)
            dic[key] = [obj["lexical_complexity_score"]]
        # Transform dictionary into 
        dic = {k : np.mean(dic[k]) for k in dic.keys()}
        response["content"] = dic          
      except Exception as e:
        response["message"] = 'Error: {}'.format(str(e))
    else:
      response["message"] = "Inexisting parameter!"  
  else:
    response["message"] = "No other resources available yet..."  
  headers = {
    "Access-Control-Allow-Origin": "*",
    'Access-Control-Allow-Headers': 'Content-Type'
  }
  return (json.dumps(response['content'], ensure_ascii=False), 200, headers)

In [136]:
request = {
  "resource": "get_average_lexical_complexity",
  "min": 1950,
  "max": 1960,
  "timestamp": 5
}
test = get_avg_complexity_by_year_in_interval(request)

In [137]:
test[0]

'{"1950,1955": 0.6212121333333332, "1957,1960": 0.6100264828571429}'

In [131]:
test[0]

'{"1950,1955": 0.6212121333333332, "1957,1960": 0.6100264828571429}'

In [114]:
test[0]

'{"1950,1954": [{"song_title": "Não tem solução", "artist": "Dorival Caymmi", "release_date": 1952, "rank_position": 22, "lexical_complexity_score": 0.6818182, "mxm_dataset_ID": "TRPDNEY128F1472E46"}, {"song_title": "Sebastiana", "artist": "Jackson do Pandeiro", "release_date": 1953, "rank_position": 58, "lexical_complexity_score": 0.6666667, "mxm_dataset_ID": "TRJILUW128F92F8BFF"}], "1955,1959": [{"song_title": "Saudosa Maloca", "artist": "Demônios da Garoa", "release_date": 1955, "rank_position": 2, "lexical_complexity_score": 0.5151515, "mxm_dataset_ID": "TRCIOEX12903CB102E"}, {"song_title": "Saudade da Bahia", "artist": "Dorival Caymmi", "release_date": 1957, "rank_position": 12, "lexical_complexity_score": 0.53333336, "mxm_dataset_ID": "TRUATRA128F1475223"}, {"song_title": "Franqueza", "artist": "Maysa", "release_date": 1957, "rank_position": 18, "lexical_complexity_score": 0.7105263, "mxm_dataset_ID": "TRPYNGY12903CE2EA7"}, {"song_title": "Estrada do sol", "artist": "Agostinho do

In [111]:
test[0]

'{"1950,1954": [{"song_title": "Não tem solução", "artist": "Dorival Caymmi", "release_date": 1952, "rank_position": 22, "lexical_complexity_score": 0.6818182, "mxm_dataset_ID": "TRPDNEY128F1472E46"}, {"song_title": "Sebastiana", "artist": "Jackson do Pandeiro", "release_date": 1953, "rank_position": 58, "lexical_complexity_score": 0.6666667, "mxm_dataset_ID": "TRJILUW128F92F8BFF"}], "1955,1959": [{"song_title": "Saudosa Maloca", "artist": "Demônios da Garoa", "release_date": 1955, "rank_position": 2, "lexical_complexity_score": 0.5151515, "mxm_dataset_ID": "TRCIOEX12903CB102E"}, {"song_title": "Saudade da Bahia", "artist": "Dorival Caymmi", "release_date": 1957, "rank_position": 12, "lexical_complexity_score": 0.53333336, "mxm_dataset_ID": "TRUATRA128F1475223"}, {"song_title": "Franqueza", "artist": "Maysa", "release_date": 1957, "rank_position": 18, "lexical_complexity_score": 0.7105263, "mxm_dataset_ID": "TRPYNGY12903CE2EA7"}, {"song_title": "Estrada do sol", "artist": "Agostinho do

In [96]:
test[0]

'{"1950,1953": [0.6818182, 0.6666667], "1955,1958": [0.5151515, 0.53333336, 0.7105263, 0.56666666], "1959,1962": [0.59090906, 0.71875, 0.4, 0.75]}'

In [ ]:
url = "https://split-by-interval-xg7jaquagq-rj.a.run.app"

payload = json.dumps({
  "resource": "get_average_lexical_complexity",
  "min": 1950,
  "max": 1960,
  "timestamp": 2
})
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)